In [1]:
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
import numpy as np
sys.path.append("..")

In [6]:
from sklearn.metrics import average_precision_score,precision_recall_curve
import matplotlib.pyplot as plt
from compute_score import calc_true_and_false_positives_for_an_example
from box_processing.get_boxes_from_net_output import get_all_pred_boxes, get_all_gt_boxes

importing Jupyter notebook from compute_score.ipynb


In [3]:
classes = ["TD", "TC", "ETC", "AR"]
class BBox_Accuracy(object):
    def __init__(self, num_classes=4, iou_thresh=0.5, reverse_conf=False, use_pascal_voc=False):
        self.use_pascal_voc = use_pascal_voc
        self.iou_thresh = iou_thresh
        self.num_classes = num_classes
        self.reverse_conf = reverse_conf
        self.y_score ={cls:[] for cls in range(num_classes)}
        self.y_true = {cls:[] for cls in range(num_classes)}
        
    def update_scores(self,net_output, label):
        self._compute_score(net_output, label)

    def _compute_score(self,net_output, label):
        pred_boxes = get_all_pred_boxes(net_output)
        gt_boxes = get_all_gt_boxes(label)
        for ex_ind in range(net_output.shape[0]):
            for cls in range(self.num_classes):
                y_true, y_score = calc_true_and_false_positives_for_an_example(pred_boxes[ex_ind],
                                                                                 gt_boxes[ex_ind],
                                                                                 cls,
                                                                                 self.iou_thresh, reverse_conf=self.reverse_conf)
                self.y_score[cls].extend(y_score)
                self.y_true[cls].extend(y_true)


    def compute_final_accuracy(self):
        APs = self.compute_final_APs()
        mAP = self.compute_mAP(APs)
        return mAP


    def compute_mAP(self,APs):
        filtered_aps = [ap for ap in APs.values() if ap>=0.0]
        if len(filtered_aps) > 0:
            return np.mean(filtered_aps)
        else:
            return 0.0

    def compute_final_APs(self):
        APs = {}
        for cls in self.y_score.keys():
            scores = self.y_score[cls]
            trues = self.y_true[cls]
            if len(self.y_score[cls]) > 0:
                if np.any(scores):
                    
                    ap = compute_average_precision(trues, scores,use_pascal_voc=self.use_pascal_voc)
                    if not np.isnan(ap):
                        APs[cls] = ap
                    else:
                        APs[cls] = 0.0
                    
                else:
                    APs[cls] = -1
                
            else:
                APs[cls] = -1
            
            print "Score for cls %s is %8.4f" % (classes[cls], APs[cls])
        return APs

    

In [35]:
def compute_average_precision(trues, scores, use_pascal_voc=False):
    if use_pascal_voc:
        raise NotImplementedError
        
        
    
    else:
        ap =  average_precision_score(trues, scores)
    return ap
        

In [ ]:
#! jupyter nbconvert --to script accuracy.ipynb

In [ ]:
#! cp accuracy.py /project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup